In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option("display.max_columns",None)

In [ ]:
df = pd.read_csv("../Dataset/gurgaon_properties_outlier_treated.csv")

In [ ]:
df.head()

In [ ]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
df.isnull().sum()

Observation
* built_up_area has highest missing value, so we will impute that column first.
* Approach is check super built up area and carpet area and find its ration and fill it with that ratio. so we will only keep built up area 

## Built up area

In [ ]:
sns.scatterplot(x = df['built_up_area'],y = df['super_built_up_area'])

In [ ]:
sns.scatterplot(x = df['built_up_area'],y = df['carpet_area'])

In [ ]:
carpet_to_built_up_ratio = round((df['carpet_area'].dropna()/df['built_up_area'].dropna()).median(),1)
print(carpet_to_built_up_ratio)

In [ ]:
super_to_built_up_ratio = round((df['super_built_up_area'].dropna()/df['built_up_area'].dropna()).median(),2)
print(super_to_built_up_ratio)

In [ ]:
# only built up area is null
built_null_df = df[((~df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (~df['carpet_area'].isnull()))]

In [ ]:
built_null_df.head()

In [ ]:
built_null_df['built_up_area'] = round(
    ((built_null_df['super_built_up_area']/super_to_built_up_ratio) + (built_null_df['carpet_area']/carpet_to_built_up_ratio))/2,2)

In [ ]:
built_null_df.head()

In [ ]:
df.update(built_null_df)

In [ ]:
df.isnull().sum()

In [ ]:
# both built up area and carpet area is null and super built up is not null
b_c_null_df = df[~(df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull())]

In [ ]:
b_c_null_df.head()

In [ ]:
b_c_null_df['built_up_area'] = round(b_c_null_df['super_built_up_area']/super_to_built_up_ratio,2)

In [ ]:
b_c_null_df.head()

In [ ]:
df.update(b_c_null_df)

In [ ]:
df.isnull().sum()

In [ ]:
# both super built up and built up is null and carpet is not null
s_b_null_df = df[(df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (~df['carpet_area'].isnull())]

In [ ]:
s_b_null_df.head()

In [ ]:
s_b_null_df['built_up_area'] = round((s_b_null_df['carpet_area']/carpet_to_built_up_ratio),2)

In [ ]:
s_b_null_df.head()

In [ ]:
df.update(s_b_null_df)

In [ ]:
df.isnull().sum()

In [ ]:
sns.scatterplot(x = df['built_up_area'],y = df['price'])

Observation
* All the filling has been done and the built up area to price ration should be linear but no idea why i am getting this

In [ ]:
anamoly_df = df[(df['built_up_area'] < 2000) & (df['price'] > 2.5)][['price','area','built_up_area']]
anamoly_df.sample(5)

In [ ]:
anamoly_df['built_up_area'] = anamoly_df['area']

In [ ]:
df.update(anamoly_df)

In [ ]:
sns.scatterplot(x = df['built_up_area'],y = df['price'])

Observation
* Filled the built up area 

In [ ]:
df.drop(columns=['area','areaWithType','super_built_up_area','carpet_area','area_room_ratio'],inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

## facing

In [ ]:
1009/df.shape[0]

In [ ]:
df['facing'].value_counts().plot(kind='pie',autopct='%0.2f%%')

In [ ]:
df.drop(columns=['facing'],inplace=True)

In [ ]:
df.isnull().sum()

Observation
* Facing has lot of missing value.
* from pie chart i am not get how to fill the null value there might be a relation but don't know.
* finally i am droping that column


## floornum

In [ ]:
df[df['floorNum'].isnull()]

In [ ]:
df[df['property_type'] == 'house']['floorNum'].median()

In [ ]:
df['floorNum'].fillna(2.0,inplace=True)

In [ ]:
df.isnull().sum()

Observation
* in floor num all are hosue only 2 are flat so replacing the null value with the median of house

## society

In [ ]:
df[df['society'].isnull()]

In [ ]:
df.drop(index=[2439],inplace=True)

## agePossession

Observation
* age posession has lot of undefined value and need to fill it
* we will go to secto and proerty type and check what are matching with undefined and will fill it with mode value of those matched result.
* By checking both still there was undefined because the combination of sector and proeprty type gives null so we pick solo columns and filled the null values.

In [ ]:
df['agePossession'].value_counts()

In [ ]:
df[df['agePossession'] == 'Undefined']

In [ ]:
def mode_based_imputation(row):
    if row['agePossession'] == 'Undefined':
        mode_value = df[(df['sector'] == row['sector']) & (df['property_type'] == row['property_type'])]['agePossession'].mode()
        # If mode_value is empty (no mode found), return NaN, otherwise return the mode
        if not mode_value.empty:
            return mode_value.iloc[0] 
        else:
            return np.nan
    else:
        return row['agePossession']

In [ ]:
df['agePossession'] = df.apply(mode_based_imputation,axis=1)

In [ ]:
df['agePossession'].value_counts()

In [ ]:
def mode_based_imputation2(row):
    if row['agePossession'] == 'Undefined':
        mode_value = df[(df['sector'] == row['sector'])]['agePossession'].mode()
        # If mode_value is empty (no mode found), return NaN, otherwise return the mode
        if not mode_value.empty:
            return mode_value.iloc[0] 
        else:
            return np.nan
    else:
        return row['agePossession']

In [ ]:
df['agePossession'] = df.apply(mode_based_imputation2,axis=1)

In [ ]:
df['agePossession'].value_counts()

In [ ]:
def mode_based_imputation3(row):
    if row['agePossession'] == 'Undefined':
        mode_value = df[(df['property_type'] == row['property_type'])]['agePossession'].mode()
        # If mode_value is empty (no mode found), return NaN, otherwise return the mode
        if not mode_value.empty:
            return mode_value.iloc[0] 
        else:
            return np.nan
    else:
        return row['agePossession']

In [ ]:
df['agePossession'] = df.apply(mode_based_imputation3,axis=1)


In [ ]:
df['agePossession'].value_counts()

In [ ]:
df.shape

In [ ]:
df.to_csv('../Dataset/gurgaon_properties_missing_value_imputation.csv',index=False)